# Example usages of **chat** and **service** interfaces
This notebook shows simple usages of MemoryScope's **chat** and **service** interfaces, along with its main features.

Before running this notebook, follow the [**Installation**](../../docs/installation.md#iii-install-from-pypi) guidelines in Readme, and start the Docker image first.

## Initiate a MemoryScope instance
First, we need to specify a configuration and initiate a MemoryScope instance.

In [ ]:
from memoryscope import MemoryScope, Arguments
arguments = Arguments(
    language="en",
    human_name="User",
    assistant_name="AI",
    memory_chat_class="api_memory_chat",
    generation_backend="dashscope_generation",
    generation_model="qwen2-72b-instruct",
    embedding_backend="dashscope_embedding",
    embedding_model="text-embedding-v2",
    rank_backend="dashscope_rank",
    rank_model="gte-rerank",
    enable_ranker=True,
    worker_params={"get_reflection_subject": {"reflect_num_questions": 3}}
)

ms = MemoryScope(arguments=arguments)


## Chat without memory
MemoryScope comes with a default **chat** interface, so it's very easy to start chatting, just as what you'll do with any LLM chatbot.

In [ ]:
memory_chat = ms.default_memory_chat
memory_chat.run_service_operation("delete_all")
response = memory_chat.chat_with_memory(query="My hobby is to play piano.")
print("Response 1: \n" + response.message.content)

Response 1: 
That's wonderful! Playing the piano is a beautiful and expressive hobby that can bring joy, relaxation, and a sense of achievement. It engages both the creative and technical aspects of your mind, enhancing cognitive skills and fostering emotional expression. Whether you enjoy classical pieces, modern compositions, or improvisation, the piano offers a vast repertoire to explore. Keep practicing and enjoying your musical journey!


----
You can choose to chat with or without multi-round conversation contexts. However, since **Memory Consolidation** has not been called, there's no memory pieces in the system yet.

In [ ]:
response = memory_chat.chat_with_memory(query="Do you know if I play any musical instruments?")
print("Response 2: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="Do you know if I play any musical instruments?",
                                        history_message_strategy=None)
print("Response 3: \n" + response.message.content)

Response 2: 
Based on our conversation, you have mentioned that your hobby is to play the piano. Therefore, yes, you play a musical instrument – the piano.
Response 3: 
I'm sorry, but as MemoryScope, I don't have access to personal information about individuals unless it has been shared with me during our conversation. Therefore, I cannot determine if you play any musical instruments. If you do play an instrument, feel free to share that information with me!


## Memory Consolidation
Now, we do a bit more chatting and then try out **Memory Consolidation**.

In [ ]:
response = memory_chat.chat_with_memory(query="I work for Meta")
print("Response 4: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="What fruit should I eat this afternoon?")
print("Response 5: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="I like watermelon。")
print("Response 6: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="Help me write a short birthday wish for a friend.")
print("Response 7: \n" + response.message.content)

Response 4: 
Thank you for letting me know! Working for Meta (formerly known as Facebook) implies that you're part of a company at the forefront of technology, social media, and innovation. Meta is known for its influential platforms like Facebook, Instagram, and WhatsApp, as well as its work in virtual reality with Oculus and efforts towards building the metaverse. It's exciting to be part of a company shaping the future of digital communication and interaction.
Response 5: 
Eating fruit is a great way to nourish your body and satisfy your taste buds. For this afternoon, consider having a refreshing and nutritious option like a juicy orange, a sweet apple, a handful of berries (such as strawberries, blueberries, or raspberries), or a slice of refreshing watermelon. These fruits are not only delicious but also packed with vitamins, antioxidants, and fiber to keep you energized throughout the day. Choose the one that appeals to you most or mix a few for a colorful fruit salad!
Response 

In [ ]:
memory_service = ms.default_memory_service
memory_service.init_service()
result = memory_service.consolidate_memory()
print(f"consolidate_memory result={result}")

consolidate_memory result=[MEMORY ACTIONS]:
new observation: user's hobby is playing the piano (valid)
new observation: user works for meta (valid)
new observation: user likes watermelon (valid)


----
**Memory Consolidation** extracted 3 *observations* out of the 7 chat messages from the user, with the uninformative ones being filtered out.

We try more cases to test its time awareness and the ability to filter out fictitious contents from the user.

In [ ]:
response = memory_chat.chat_with_memory(query="What are the prospects like if I go to work at Amazon?")
print("Response 8: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="Note: I am planning a business trip to Seattle next week.")
print("Response 9: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="My classmate Liam is currently working at Google. He will return to New York next month, and I plan to have a meal with him.")
print("Response 10: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="Cynthia is my best friend, and she has decided to go to university in Iowa. Write an 80-word micro drama starting with this.")
print("Response 11: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="What kind of company is SMCI, and what do they do?")
print("Response 12: \n" + response.message.content)

Response 8: 
Prospects at Amazon can be quite promising, given its status as one of the world's leading tech companies. You can expect competitive compensation, career growth opportunities, a dynamic work environment, and the chance to work on innovative projects. However, actual experiences may vary based on role, team, and individual performance. It's always a good idea to research specific positions, company culture, and employee reviews to get a more comprehensive understanding.
Response 9: 
Great, planning a business trip to Seattle can be exciting. Some key points to consider for your trip:

1. **Weather**: Check the weather forecast for Seattle to pack appropriately. Seattle is known for its mild, marine climate with possible rainfall throughout the year.

2. **Transportation**: Familiarize yourself with transportation options like Sea-Tac Airport, light rail, buses, and ride-sharing services for getting around.

3. **Accommodation**: Book a hotel near your meeting location or i

In [ ]:
result = memory_service.consolidate_memory()
print(f"consolidate_memory result={result}")

consolidate_memory result=[MEMORY ACTIONS]:
new observation: user is interested in career prospects at amazon (valid)
new observation: user is inquiring about smci's nature and business (valid)
new observation: user is planning a business trip to seattle next week (Inference time: next week after august 2, 2024) (valid)
new observation: user plans to meet classmate liam in new york next month (Inference time: next month after august 2, 2024) (valid)


----
We can see **Memory Consolidation** successfully filtered out fictitious contents, and shows good time sensitivity.

We try more cases to test its resolution of conflicting contents.

In [ ]:
response = memory_chat.chat_with_memory(query="What fruit should I eat this afternoon?")
print("Response 13: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="Watermelon is really good, but I also like mangoes. Today, I want to eat mangoes.")
print("Response 14: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="I recently switched jobs and joined Apple.")
print("Response 15: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="I also like eating peaches and apples.")
print("Response 16: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="I don't like eating coconuts.")
print("Response 17: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="I am planning to go surfing in Florida next month.")
print("Response 18: \n" + response.message.content)
response = memory_chat.chat_with_memory(query="Tomorrow is my birthday")
print("Response 19：\n" + response.message.content)

Response 13: 
You might enjoy eating watermelon this afternoon since you like it.
Response 14: 
Great choice! Enjoy your mangoes today.
Response 15: 
Congratulations on your new job at Apple! How are you finding the experience so far?
Response 16: 
That's wonderful! Peaches and apples are both delicious and nutritious choices. Enjoy snacking on them whenever you like.
Response 17: 
No problem at all, everyone has their preferences. If you don't like coconuts, there are plenty of other fruits to enjoy.
Response 18: 
Sounds like an exciting plan! Florida offers great surfing spots with its warm waters and waves. Have a fantastic time surfing next month.
Response 19：
Happy Birthday for tomorrow! May your day be filled with joy, laughter, and memorable moments.


In [ ]:
result = memory_service.consolidate_memory()
print(f"consolidate_memory result={result}")

consolidate_memory result=[MEMORY ACTIONS]:
new observation: user likes mangoes and wants to eat them today (valid)
new observation: user is planning to go surfing in florida next month (valid)
new observation: user's birthday is tomorrow (valid)
new observation: user recently switched jobs and joined apple (valid)
new observation: user likes eating peaches and apples (valid)
new observation: user doesn't like eating coconuts (valid)


## Reflection and Re-Consolidation
Now, we have accumulated enough new *observations* in the system, so we can call **Reflection and Re-Consolidation**, let's see what will it get.

In [ ]:
result = memory_service.reflect_and_reconsolidate()
print(f"consolidate_memory result={result}")

consolidate_memory result=[MEMORY ACTIONS]:
new insight: The Travel plans of User: travel plans include a business trip to seattle next week after august 2, 2024, meeting a classmate in new york next month after august 2, 2024, and going surfing in florida next month (valid)
new insight: The Fruit preferences of User: fruit preferences: watermelon, peaches, apples, mangoes; dislikes coconuts (valid)
new insight: The Career interest of User: amazon (valid)
modified observation: user is planning a business trip to seattle next week (Inference time: next week after august 2, 2024) (valid)
modified observation: user likes mangoes and wants to eat them today (valid)
modified observation: user is interested in career prospects at amazon (valid)
modified observation: user is inquiring about smci's nature and business (valid)
modified observation: user recently switched jobs and joined apple (valid)
modified observation: user's hobby is playing the piano (valid)
modified observation: user is p

## Low response-time (RT) for the user
Finally, we test the RT of MemoryScope system for the user. Specifically, we test the difference of RT when responding with and without retrieving memory pieces from the system.

In [ ]:
import time

start_time = time.time()
response = memory_chat.chat_with_memory(query="Do you know if I play any musical instruments?",
                                        history_message_strategy=None)
end_time = time.time()
total_time = end_time - start_time
print("With memory retrieval\nResponse 20: \n" + response.message.content + f"\n RT: {total_time} seconds\n")

start_time = time.time()
response = memory_chat.chat_with_memory(query="Do you know if I have any plans in the next month?",
                                        history_message_strategy=None)
end_time = time.time()
total_time = end_time - start_time
print("With memory retrieval\nResponse 21: \n" + response.message.content + f"\n RT: {total_time} seconds\n")

In [ ]:
memory_chat.run_service_operation("delete_all")
start_time = time.time()
response = memory_chat.chat_with_memory(query="Do you know if I play any musical instruments?",
                                        history_message_strategy=None)
end_time = time.time()
total_time = end_time - start_time
print("Without memory retrieval\nResponse 22: \n" + response.message.content + f"\n RT: {total_time} seconds\n")

start_time = time.time()
response = memory_chat.chat_with_memory(query="Do you know if I have any plans in the next month?\n",
                                        history_message_strategy=None)
end_time = time.time()
total_time = end_time - start_time
print("Without memory retrieval\nResponse 23: \n" + response.message.content + f"\n RT: {total_time} seconds\n")

With memory retrieval
Response 20: 
Yes, you play the piano.
 RT: 1.494797706604004 seconds

With memory retrieval
Response 21: 
Yes, you have the following plans in the next month:

1. A business trip to Seattle sometime after August 2, 2024.
2. Meeting a classmate in New York next month, also after August 2, 2024.
3. Going surfing in Florida next month.
 RT: 6.400442123413086 seconds

Without memory retrieval
Response 22: 
I'm sorry, but as MemoryScope, I don't have access to personal information about individuals unless it has been shared with me during our conversation. Therefore, I cannot determine if you play any musical instruments without you providing that information. Have you mentioned anything about your musical abilities before?
 RT: 3.8412117958068848 seconds

Without memory retrieval
Response 23: 
I'm sorry, but as an AI, I don't have access to personal schedules or information unless it has been shared with me during our conversation, which we haven't had. I cannot acce

----
We can see responding with retrieving memory pieces from MemoryScope does not increase RT.

## More Examples
We direct the reader to [Advanced Customization](../advance/custom_operator.md) for guidance on customizing the various settings of the MemoryScope system. It is also possible to create or customize your own MemoryScope **operations** by specifying a **workflow** and the corresponding **workers** that best meet your specific needs.

Additionally, you can also try out the [CLI with a MemoryScope Chatbot](../cli/CLI_README.md). We have implemented the chatbot so that the **Memory Consolidation** and **Reflection and Re-Consolidation** operations are always run asynchronously in the backend, ensuring that they do not incur any response time for the user.

